In [1]:
#Importar librería gbq
import pandas as pd
import itertools
from google.cloud import bigquery



In [2]:
# Autenticación
client = bigquery.Client()

# crear archivo
archivo = '../../../Correos shows Mailing/Catalogo_mailing_shows.xlsx'
excel_writer = pd.ExcelWriter(archivo, engine="xlsxwriter")

lista_shows = [
    'rutasecreta',
    'historiafreak',
    'cocavi',
    'eneup',
    'oficio']

nombre_rdf = {
    "rutasecreta": "La Ruta Secreta", 
    "historiafreak" : "Historia Freak",
    "cocavi": "Cocavi", 
    "eneup" : "Esto No Es un Podcast", 
    "oficio": "Oficio de vivir"}


In [3]:
for show in range(len(lista_shows)):

    query_shows = """

    with consumo as (
    SELECT distinct dicc_shows._id, dicc_shows.title, usuarios_unicos.email
    FROM `conexion-datos-rdf.usuarios.usuarios_emisor` AS usuarios_unicos
    LEFT JOIN`conexion-datos-rdf.consumo_usuarios.consumo_mensual_content` AS consumo_usuarios
    ON usuarios_unicos.platform_id = consumo_usuarios.user_id
    LEFT JOIN `conexion-datos-rdf.dicc_medios.dicc_medios` AS dicc_medios
    ON consumo_usuarios.content_id = dicc_medios.media_id
    LEFT JOIN `conexion-datos-rdf.diccionarios.dicc_shows` AS dicc_shows
    ON dicc_medios.show_id = dicc_shows._id), 

    rutasecreta as (
    Select * from consumo
    where consumo._id in ('5cc0d1173477b8633b077e41')),

    historiafreak as (
    Select consumo.* from consumo
    left outer join rutasecreta
    on consumo.email = rutasecreta.email
    where rutasecreta.email is null
    and consumo._id in ('5f2ac786f815b70768cc3228')),

    cocavi as (
    Select consumo.* from consumo
    left outer join rutasecreta
    on consumo.email = rutasecreta.email
    left outer join historiafreak
    on consumo.email = historiafreak.email
    where rutasecreta.email is null and historiafreak.email is null
    and consumo._id in ('62ceef3287bd310848586f38')
    ),

    eneup as (
    Select consumo.* from consumo
    left outer join rutasecreta
    on consumo.email = rutasecreta.email
    left outer join historiafreak
    on consumo.email = historiafreak.email
    left outer join cocavi
    on consumo.email = cocavi.email
    where rutasecreta.email is null and historiafreak.email is null
    and cocavi.email is null
    and consumo._id in ('5e3825c34948a26cf6148327')),

    oficio as (
    Select consumo.* from consumo
    left outer join rutasecreta
    on consumo.email = rutasecreta.email
    left outer join historiafreak
    on consumo.email = historiafreak.email
    left outer join cocavi
    on consumo.email = cocavi.email
    left outer join eneup
    on consumo.email = eneup.email
    where rutasecreta.email is null and historiafreak.email is null
    and cocavi.email is null and eneup.email is null
    and consumo._id in ('61d6f193fbdf5308349f10d5'))

    select email from {0}

    """.format(lista_shows[show])

    #print(query_shows)

    df_shows = client.query(query_shows).result().to_dataframe( create_bqstorage_client=False)  
    df_shows["show"] = nombre_rdf[lista_shows[show]]
    df_shows.to_excel(excel_writer, sheet_name=lista_shows[show], index=False)

In [4]:
excel_writer.save()